In [1]:
import pandas as pd
import math

# Чтение данных из файла
data = pd.read_csv('data_big.csv')

# Выбор нужных признаков и целевой переменной
metric_features = ['SBP.1', 'DBP.1', 'SV.1', 'CO.1']
ordinal_feature = 'weakness.1'

# Функция для разделения данных на признаки и целевую переменную
def split_data(data, metric_features, ordinal_feature):
    features = data[metric_features].values.tolist()
    target = data[ordinal_feature].values.tolist()
    return features, target

# Функция для расчета энтропии
def entropy(data):
    classes = list(set(data))
    class_counts = [data.count(cls) for cls in classes]
    total = sum(class_counts)
    entropy_val = 0
    for count in class_counts:
        probability = count / total
        entropy_val -= probability * math.log2(probability)
    return entropy_val


def cluster_split(features, ordinal_feature, num_clusters):
    # Функция для разбиения данных на группы на основе порядковых переменных
    ordinal_values = sorted(list(set(ordinal_feature)))  # Уникальные значения порядковой переменной
    cluster_diversities = []
    information_gains = []
    for i in range(num_clusters):
        if i == 0:
            # Формирование группы, где порядковая переменная меньше или равна текущему значению
            group = [row for row, ord_value in zip(features, ordinal_feature) if ord_value <= ordinal_values[i]]
        else:
            # Формирование группы, где порядковая переменная находится в интервале между двумя значениями
            group = [row for row, ord_value in zip(features, ordinal_feature) if ordinal_values[i-1] < ord_value <= ordinal_values[i]]
        cluster_diversity = len(group)  # Вычисление размера группы
        cluster_diversities.append(cluster_diversity)
        # Вычисление информационного выигрыша при разбиении групп
        information_gain_val = information_gain([range(len(group)), range(len(group), len(features))], target)
        information_gains.append(information_gain_val)
    return ordinal_values[:num_clusters], cluster_diversities, information_gains

def split_data_on_threshold(feature, threshold):
    # Функция для разделения данных на две группы по пороговому значению
    left_group = []
    right_group = []
    for i in range(len(feature)):
        if feature[i] <= threshold:
            left_group.append(i)  # Добавление индекса объекта в левую группу
        else:
            right_group.append(i)  # Добавление индекса объекта в правую группу
    return left_group, right_group

# Функция для вычисления информационного выигрыша при разбиении данных
def information_gain(groups, target):
    total_size = len(target)
    groups_entropy = 0
    for group in groups:
        group_size = len(group)
        group_entropy = entropy([target[i] for i in group])
        groups_entropy += group_size / total_size * group_entropy
    return entropy(target) - groups_entropy

# Функция для построения дерева классификации
def build_decision_tree(features, target):
    if len(set(target)) == 1:
        return {'class': target[0]}
    best_gain = 0  # Инициализация переменной для отслеживания лучшего информационного выигрыша
    best_feature_index = -1  # Инициализация индекса лучшего признака
    best_threshold = None  # Инициализация лучшего порогового значения
    best_groups = None  # Инициализация лучших групп данных
    for i in range(len(features[0])):
        feature = [row[i] for row in features]  # Получение значения признака для каждого объекта
        unique_values = list(set(feature))  # Получение уникальных значений признака
        for j in range(len(unique_values)):
            threshold = unique_values[j]  # Выбор порогового значения
            groups = split_data_on_threshold(feature, threshold)  # Разбиение данных на группы по пороговому значению
            gain = information_gain(groups, target)  # Вычисление информационного выигрыша
            if gain > best_gain:  # Проверка, является ли текущий информационный выигрыш лучшим
                best_gain = gain  # Обновление лучшего информационного выигрыша
                best_feature_index = i  # Обновление индекса лучшего признака
                best_threshold = threshold  # Обновление лучшего порогового значения
                best_groups = groups  # Обновление лучших групп данных
    if best_gain == 0:
        return {'class': max(set(target), key=target.count)}  # Возвращение класса с наибольшим количеством объектов, если информационный выигрыш равен нулю
    # Рекурсивное построение левого и правого поддеревьев
    left_child = build_decision_tree([features[i] for i in best_groups[0]], [target[i] for i in best_groups[0]])
    right_child = build_decision_tree([features[i] for i in best_groups[1]], [target[i] for i in best_groups[1]])
    # Возвращение узла дерева с лучшим признаком, пороговым значением и поддеревьями
    return {'feature_index': best_feature_index, 'threshold': best_threshold, 'left': left_child, 'right': right_child}


# Разделение данных на признаки и целевую переменную
features, target = split_data(data, metric_features, ordinal_feature)

# Построение дерева классификации
decision_tree = build_decision_tree(features, target)

# Разбиение на два кластера на основе порядковых переменных
num_clusters = 2
ordinal_values, cluster_diversities, information_gains = cluster_split(features, target, num_clusters)

# Вывод результатов для двух кластеров
print(f"Разнообразия для {num_clusters} кластеров:")
for value, diversity in zip(ordinal_values, cluster_diversities):
    print(f"Значение порядковой переменной: {value}, Разнообразие: {diversity}")
print("\nИнформационные выигрыши от объединения кластеров:")
for i in range(len(ordinal_values)-1):
    gain = information_gains[i]
    print(f"Объединение кластеров {i+1} и {i+2}: {gain}")

# Разбиение на три кластера на основе порядковых переменных
num_clusters = 3
ordinal_values, cluster_diversities, information_gains = cluster_split(features, ordinal_feature, num_clusters)

# Вывод результатов для трех кластеров
print(f"\nРазнообразия для {num_clusters} кластеров:")
for value, diversity in zip(ordinal_values, cluster_diversities):
    print(f"Значение порядковой переменной: {value}, Разнообразие: {diversity}")
print("\nИнформационные выигрыши от объединения кластеров:")
for i in range(len(ordinal_values)-1):
    gain = information_gains[i]
    print(f"Объединение кластеров {i+1}-{i+2}-{i+3}: {gain}")

Разнообразия для 2 кластеров:
Значение порядковой переменной: 0, Разнообразие: 3
Значение порядковой переменной: 1, Разнообразие: 15

Информационные выигрыши от объединения кластеров:
Объединение кластеров 1 и 2: 0.021896465123210085

Разнообразия для 3 кластеров:
Значение порядковой переменной: ., Разнообразие: 1
Значение порядковой переменной: 1, Разнообразие: 1
Значение порядковой переменной: a, Разнообразие: 1

Информационные выигрыши от объединения кластеров:
Объединение кластеров 1-2-3: 0.03553943054003694
Объединение кластеров 2-3-4: 0.03553943054003694
